In [2]:
import numpy as np

full_data = np.load('./full_data.npy', allow_pickle=True)
X_train_event, X_test_event, X_train_obj, X_test_obj, y_train, y_test = full_data

In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt



2022-02-01 10:51:29.423133: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
# all_devices = len(tf.config.list_physical_devices('GPU'))
# print("Num GPUs Available: ", all_devices)
# physical_devices=tf.config.list_physical_devices('GPU')
# gpus= tf.config.experimental.list_physical_devices('GPU')
# for i in range(0,all_devices):
#     tf.config.experimental.set_memory_growth(gpus[i], True)

# mirrored_strategy = tf.distribute.MirroredStrategy(devices=[f"/GPU:{GPU_id}" for GPU_id in range (0,6)])


Num GPUs Available:  1
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5')


2022-01-31 16:34:25.105941: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-31 16:34:25.116320: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-01-31 16:34:25.116365: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-31 16:34:25.116388: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-31 16:34:25.116398: I tensorflow/stream_executor/platform/def

In [3]:
from keras.layers import Input, LSTM, Dense, Concatenate, Flatten, Dropout
from keras.models import Model, Sequential

def build_model(hp):
    """
    Paramaters to tune:
    - Number of units per LSTM layer
    - No of LSTM layers
    - No of units per feedforward hidden layer
    - No of feedforward hidden layers
    - Activation functions for feedforward (RELU vs elu vs selu)
    - Batch normalisation 
    - Drop out
    - Optmiser (sgd vs Adam vs Nadam)

    """
    obj_input = Input(shape=X_train_obj.shape[1:])
    prev = obj_input
    for _ in range(hp.Int("LSTM layers", 1, 4, 1)-1):
        prev = LSTM(hp.Int("LSTM units", 16, 64, 16), return_sequences=True)(prev)
    lstm_out = LSTM(hp.Int("LSTM units", 16, 64, 16))(prev)
    event_input = Input(shape=X_train_event.shape[1])
    x = Concatenate()([lstm_out, event_input])
    prev = x 
    activation = hp.Choice("activation", ["relu", "selu"])
    for _ in range(hp.Int("Feedforward layers", 2, 5, 1)):
        prev = Dense(hp.Int("Feedfoward units", 100, 400, 100), activation=activation)(prev)
        if activation == "selu":
            prev.kernel_initializer = keras.initializers.LecunNormal()
        if hp.Boolean("dropout"):
            prev = Dropout(rate=0.2)(prev)
    out = Dense(1, activation="sigmoid")(prev)
    model = Model(inputs=[obj_input, event_input], outputs=out)
    model.compile(loss="binary_crossentropy", optimizer=hp.Choice("optimiser", ["Adam", "Nadam"]), metrics=[keras.metrics.AUC(), keras.metrics.Precision(), keras.metrics.Recall()])  
    return model



        
        







In [4]:
tuner = kt.RandomSearch(build_model, objective='val_loss', max_trials=20, overwrite=True)

2022-01-31 18:39:38.139779: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-31 18:39:38.177883: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/grid.cern.ch/centos7-ui-4.0.3-1_umd4v3/usr/lib64:/cvmfs/grid.cern.ch/centos7-ui-4.0.3-1_umd4v3/usr/lib:/cvmfs/grid.cern.ch/centos7-ui-4.0.3-1_umd4v3/usr/lib64:/cvmfs/grid.cern.ch/centos7-ui-4.0.3-1_umd4v3/usr/lib
2022-01-31 18:39:38.177932: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-31 18:39:38.177957: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sc01.dice.priv): /proc/driver/nvidia/version does not exist
2022-01-31 18:39:38.188298: I tensorflow/core/platform/cpu_feature_guard.cc:

In [4]:
# Imbalanced dataset so want to adjusts weights of signal and background training examples
tot = len(y_train)
pos = np.sum(y_train == 1)
neg = tot - pos
print(f'Total training samples:  {tot}\npositives:  {pos}\nnegatives:  {neg}')

# weight positives more than negatives
weight_for_0 = (1 / neg) * (tot / 2.0)
weight_for_1 = (1 / pos) * (tot / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
print(f'Postive weight:  {weight_for_1} \nNegative weight:  {weight_for_0}')

Total training samples:  327177
positives:  79444
negatives:  247733
Postive weight:  2.0591674638739237 
Negative weight:  0.660341981084474


In [6]:
tuner.search([X_train_obj, X_train_event], y_train, epochs=20, validation_data=([X_test_obj, X_test_event], y_test), class_weight=class_weight)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
LSTM layers       |2                 |?                 
LSTM units        |48                |?                 
activation        |selu              |?                 
Feedforward layers|2                 |?                 
Feedfoward units  |400               |?                 
dropout           |True              |?                 
optimiser         |Nadam             |?                 



2022-01-31 18:39:52.467646: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-31 18:39:52.540062: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2394025000 Hz


Epoch 1/20
  121/10225 [..............................] - ETA: 5:25 - loss: 0.6394 - auc: 0.7296 - precision: 0.3963 - recall: 0.6671

KeyboardInterrupt: 

In [ ]:
obj_input = Input(shape=X_train_obj.shape[1:])

